![BRONZE LOGO](https://s3-sa-east-1.amazonaws.com/arrematearte-farm/cristinagoston/lot_photos/11920/e2b04faba30f14f269a12c88c1dffc839e637495_ml.jpg)



# CAMADA BRONZE

In [0]:
from delta.tables import *
from pyspark.sql.types import *
from pyspark.sql.functions import col
import pytz
from pyspark.sql.functions import lit
from pyspark.sql import Window
from pyspark.sql.functions import *
from datetime import datetime

import requests
import json

In [0]:
try:
  time_file = datetime.now(pytz.timezone('America/Sao_Paulo')).strftime('%Y%m%d_%H%M%S')
  # DeltaTable.isDeltaTable(spark, bronze_delta_path)

  #### Nome  e caminho onde será feita a escrita da da tabela Delta  ##########
  db = 'bronze.'
  table = 'ibge_news'
  delta_table = f'{db}{table}'
  bronze_delta_path = 'dbfs:/mnt/bronze/'

  ### Sistema de origem ####
  source_system_path = 'dbfs:/mnt/raw_3/'

  ### caminho para gravação do histórico ###
  historic_path = 'dbfs:/mnt/historic/bronze/'
  
  print("##----------------------------##")
  print(f"Data e Horário de execusão             ===> {time_file} \n")
  print(f"Tabela bronze a ser criada             ===>  {delta_table}")
  print(f"Caminho da tabela Bronze a ser criada  ===> {bronze_delta_path} \n")
  print("##----------------------------##")
  print(f"Sistema de origem                      ===> {source_system_path}")
  print(f"Caminho para gravação do histórico     ===> {historic_path}")
except Exception as e:
  print(e)

##----------------------------##
Data e Horário de execusão             ===> 20241110_080740 

Tabela bronze a ser criada             ===>  bronze.ibge_news
Caminho da tabela Bronze a ser criada  ===> dbfs:/mnt/bronze/ 

##----------------------------##
Sistema de origem                      ===> dbfs:/mnt/raw_3/
Caminho para gravação do histórico     ===> dbfs:/mnt/historic/bronze/


### Listando arquivos camada RAW

### Separando Arquivos de 10 em 10

### Criando Classe de Ingestao na Bronze

In [0]:
class BronzeIngestion:

  def __init__(self,db,table,bronze_delta_path,source_system_path,historic_path,period):
    self.db                 = db
    self.table              = table
    self.delta_table        = f"{self.db}.{self.table}"
    self.bronze_delta_path  = bronze_delta_path
    self.source_system_path = source_system_path
    self.historic_path      = historic_path
    self.period             = str(period)

  ### Listando arquivos camada RAW   ####
  def list_all_files(self):
    try:
      #### Faz a comparação entre Raw e histórico caso ja existam arquivos no Historico  ####
      ################################################################################
      
      #### Lista de arquivos da Raw ####
      raw_set = {files.path.split('/')[-2] for files in dbutils.fs.ls(f'{self.source_system_path}{self.period}/')} 

      ### Lista de arquivos Historico   ####
      try:
        historic_set = {files.path.split('/')[-2]  for files in dbutils.fs.ls(f'{self.historic_path}{self.period}/')}
      except:
        ## Caso nao haja ANO e MES mais recentes no historico.
        all_files = [f'{self.source_system_path}{self.period}/{files}'  for files in list(raw_set)]
        print(f'Número de arquivos para ingestão == > {len(all_files)}')
        return all_files

      all_files = [f'{self.source_system_path}{self.period}/{files}'  for files in list(historic_set.symmetric_difference(raw_set))]
      print(all_files)
      print(f'Número de arquivos para ingestão == > {len(all_files)}')
      return all_files

    except Exception as error:
      print(f"{error}")    

 
  def separate_files(self):
    try:
      ### Separando arquivos a cada 10 items ou a quantidade que for..
      ### caso seja menor do que 10
      all_files = self.list_all_files()
      return  [ all_files[i:i+10] for i in range(0,len(all_files),10)]
    
    except Exception as error:
      print(f"{error}") 
  
  ### Salvando Dados na camada Bronze ###
  def save_files_delta(self):
    try:
      print(" Inicinado método list_all_files().... \n Inicinado método list_all_files() .... \n Inicinado método save_files_delta() ....\n")
      files_to_record = self.separate_files()
      for files in files_to_record:
        # print(files,len(files),"\n")
        df =spark.read.json(files)
        #### Adicionando data de ingestao ####
        df = df.withColumn('DTPROC',lit(datetime.now(pytz.timezone('America/Sao_Paulo')).strftime('%Y%m%d_%H%M%S')))
        print(f" Salvando dados da API no caminho ===> {self.bronze_delta_path}  com o nome ===>  {self.delta_table} ")
        df.write.mode('append').format('delta').save(self.bronze_delta_path)
      print("\n Dados salvos em formato delta. \n")
    except Exception as error:
      print(f"{error}")
  
  ##### Copiando arquivos para Historico  ####
  def copy_to_historic(self):
    try:
      lista_arquivos = self.list_all_files()
      for folder in lista_arquivos:
        name_folder = folder.split('/')[-1]
        date_folder = folder.split('/')[-2]
        print(f"Copiando arquivo {self.source_system_path +date_folder+'/'+ name_folder +'/'} para ===> {self.historic_path+date_folder+'/'}")
        dbutils.fs.cp(self.source_system_path +date_folder+'/'+name_folder,self.historic_path + date_folder +'/'+name_folder +'/',recurse = True)

    except Exception as error:
      print(f"{error}")
  
  ### Criando tabela no hive_metastore (catalogo Databricks)  ####
  def create_delta_table_hive(self):
    try:
      sql = f""" CREATE DATABASE IF NOT EXISTS {self.db} """
      spark.sql(sql)
      print(sql,"\n")

      sql_drop = f""" DROP TABLE IF EXISTS {self.delta_table}  """
      spark.sql(sql_drop)
      print(sql_drop,"\n") 

      sql_table = f""" CREATE TABLE IF NOT EXISTS {self.delta_table} USING DELTA LOCATION '{self.bronze_delta_path}' """
      spark.sql(sql_table)
      print(sql_table)

    except Exception as error:
      print(f"{error}")

  

  #### Executando  ####
  def bronze_run(self):
    print("Inicio do processo de ingestão na bronze... \n")
    # self.list_all_files()
    # self.separate_files()
    self.save_files_delta()
    self.copy_to_historic()
    self.create_delta_table_hive()
    print("Processo finalizado!..")

In [0]:
bronze = BronzeIngestion('bronze','ibge_news','dbfs:/mnt/bronze/','dbfs:/mnt/raw_3/','dbfs:/mnt/historic/bronze/',202411)
bronze.bronze_run()

Inicio do processo de ingestão na bronze... 

 Inicinado método list_all_files().... 
 Inicinado método list_all_files() .... 
 Inicinado método save_files_delta() ....

[]
Número de arquivos para ingestão == > 0

 Dados salvos em formato delta. 

[]
Número de arquivos para ingestão == > 0
 CREATE DATABASE IF NOT EXISTS bronze  

 DROP TABLE IF EXISTS bronze.ibge_news   

 CREATE TABLE IF NOT EXISTS bronze.ibge_news USING DELTA LOCATION 'dbfs:/mnt/bronze/' 
Processo finalizado!..


In [0]:
%sql
-- select count(*) from delta.`dbfs:/mnt/bronze/`
select * from bronze.ibge_news limit 5

PAGE,data_publicacao,destaque,editorias,id,imagens,introducao,link,produto_id,produtos,produtos_relacionados,tipo,titulo,DTPROC
11 - to - 20 - 20241026_105918,04/06/2024 09:00:00,true,economicas,40241,"{""image_intro"":""images\/agenciadenoticias\/releases\/ContasNacionais.png"",""float_intro"":"""",""image_intro_alt"":"""",""image_intro_caption"":"""",""image_fulltext"":""images\/agenciadenoticias\/releases\/ContasNacionais.png"",""float_fulltext"":"""",""image_fulltext_alt"":"""",""image_fulltext_caption"":""""}","No primeiro trimestre de 2024, o PIB cresceu 0,8% frente ao quarto trimestre de 2023, na série com ajuste sazonal. Pela ótica da produção, os destaques foram Serviços (1,4%) e Agropecuária (11,3%), enquanto a Indústria ficou estável (-0,1%). Período de...",http://agenciadenoticias.ibge.gov.br/agencia-sala-de-imprensa/2013-agencia-de-noticias/releases/40241-pib-cresce-0-8-no-primeiro-trimestre-de-2024.html,9300,9300|Sistema de Contas Nacionais Trimestrais|contas-nacionais-trimestrais|2087,9300,Release,"PIB cresce 0,8% no primeiro trimestre de 2024",20241026_111053
11 - to - 20 - 20241026_105918,04/06/2024 09:00:00,true,economicas,40242,"{""image_intro"":""images\/agenciadenoticias\/estatisticas_economicas\/2024_06\/PIB-Tri_THUMB_HelenaPontes.jpg"",""float_intro"":"""",""image_intro_alt"":"""",""image_intro_caption"":"""",""image_fulltext"":""images\/agenciadenoticias\/estatisticas_economicas\/2024_06\/PIB-Tri_HOME_HelenaPontes.jpg"",""float_fulltext"":"""",""image_fulltext_alt"":"""",""image_fulltext_caption"":""""}","Setor de serviços puxa alta do PIB no primeiro trimestre de 2024, com destaque para o comércio varejista. - Foto: Helena Pontes/Agência IBGE Notícias No primeiro trimestre de 2024, o Produto Interno Bruto (PIB) do país teve alta de 0,8% frente ao último...",http://agenciadenoticias.ibge.gov.br/agencia-noticias/2012-agencia-de-noticias/noticias/40242-pib-cresce-0-8-no-primeiro-trimestre-puxado-por-servicos.html,9300,9300|Sistema de Contas Nacionais Trimestrais|contas-nacionais-trimestrais|2087,9300,Notícia,"PIB cresce 0,8% no primeiro trimestre, puxado por Serviços",20241026_111053
11 - to - 20 - 20241026_105918,03/06/2024 05:54:52,true,ibge;censo2020,40240,"{""image_intro"":""images\/agenciadenoticias\/ibge\/2024_06\/Atendiment-THUMB.jpg"",""float_intro"":"""",""image_intro_alt"":"""",""image_intro_caption"":"""",""image_fulltext"":""images\/agenciadenoticias\/ibge\/2024_06\/Atendiment-HOME.jpg"",""float_fulltext"":"""",""image_fulltext_alt"":"""",""image_fulltext_caption"":""""}","Atendimento aos gestores no balcão do IBGE. Na foto, prefeito de Japaratinga e Superintendente da SES/AL - Foto: Nathalia Leal Um dos símbolos do litoral nordestino, a Área de Proteção Ambiental (APA) Costa dos Corais é a maior unidade de conservação...",http://agenciadenoticias.ibge.gov.br/agencia-noticias/2012-agencia-de-noticias/noticias/40240-superintendencia-do-ibge-mostra-dados-do-censo-2022-para-gestores-publicos-em-alagoas.html,0,,,Notícia,Superintendência do IBGE mostra dados do Censo 2022 para gestores públicos em Alagoas,20241026_111053
11 - to - 20 - 20241026_105918,03/06/2024 02:00:00,true,ibge,40235,"{""image_intro"":""images\/agenciadenoticias\/ibge\/2024_06\/ContadorIBGE_THUMB.jpg"",""float_intro"":"""",""image_intro_alt"":"""",""image_intro_caption"":"""",""image_fulltext"":""images\/agenciadenoticias\/ibge\/2024_06\/ContadorIBGE_HOME.jpg"",""float_fulltext"":"""",""image_fulltext_alt"":"""",""image_fulltext_caption"":""""}","O IBGE comemorou seus 88 anos, em 29 de maio de 2024, com uma cerimônia repleta de homenagens, no Palácio da Fazenda, no Rio de Janeiro (RJ). No mesmo dia, houve inauguração da Casa Brasil IBGE e consolidação de acordos de cooperação técnica e protocolos...",http://agenciadenoticias.ibge.gov.br/agencia-noticias/2012-agencia-de-noticias/noticias/40235-contagem-regressiva-para-os-90-anos-do-ibge.html,0,,,Notícia,Contagem regressiva para os 90 anos do IBGE,20241026_111053
11 - to - 20 - 20

In [0]:
# display(dbutils.fs.ls('dbfs:/mnt/historic/bronze/202410/ibgeapipage_1_to_10_20241019_110702/'))

## apagar pastas
# dbutils.fs.rm('dbfs:/mnt/raw/',True)
# dbutils.fs.rm('dbfs:/mnt/bronze/',True)
# dbutils.fs.rm('dbfs:/mnt/historic/bronze/',True)

## criar pasta
# dbutils.fs.mkdirs('dbfs:/mnt/raw/')
# dbutils.fs.mkdirs('dbfs:/mnt/raw_2/')
# dbutils.fs.mkdirs('dbfs:/mnt/raw_3/')

In [0]:
# ### é Possivel fazer a Leitura de varios arquivos de forma Multipla, passando a referencia deles dentro de uma lista  ##########

# spark.read.json(files).display()